In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spy =pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/SPY.csv',index_col ='Date')['Adj Close'])
tlt =pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/TLT.csv',index_col ='Date')['Adj Close'])
btc =pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/BTC-USD.csv',index_col ='Date')['Adj Close'])
vxx =pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/VXX.csv',index_col ='Date')['Adj Close'])
xlf=pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/XLF.csv',index_col ='Date')['Adj Close'])
fxi =pd.DataFrame(pd.read_csv('C:/Users/sherryyang/Desktop/Risk/assignment6/FXI.csv',index_col ='Date')[ 'Adj Close'])

In [3]:
combined1 = pd.merge(btc, spy, left_index=True, right_index=True, how='outer')
combined1.columns =['btc','spy']
combined2 = pd.merge(combined1, tlt, left_index=True, right_index=True, how='outer')
combined2.columns =['btc','spy', 'tlt']
combined3 = pd.merge(combined2, vxx, left_index=True, right_index=True, how='outer')
combined3.columns =['btc','spy', 'tlt','vxx']
combined4 = pd.merge(combined3, xlf, left_index=True, right_index=True, how='outer')
combined4.columns =['btc','spy', 'tlt','vxx', 'xlf']
#drop the btc adj return in weekends
combined5 = pd.merge(combined4, fxi, left_index=True, right_index=True, how='inner')
combined5.columns =['btc','spy', 'tlt','vxx', 'xlf', 'fxi']


In [4]:
combined5.head()

,btc,spy,tlt,vxx,xlf,fxi
Date,,,,,,
2015-03-09,290.019989,196.044159,115.734543,442.559998,14.388450,39.373985
2015-03-10,291.369995,192.863953,117.257355,456.160004,14.081054,38.458961
2015-03-11,295.600006,192.412338,118.111649,467.679993,14.169726,38.553299
2015-03-12,293.850006,194.858627,118.028046,439.679993,14.477120,38.940056
2015-03-13,287.209991,193.663681,117.665939,449.600006,14.376628,39.138153


In [4]:
import pandas as pd
import requests
from multiprocessing import Pool
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

class Data(object):
    def __init__(self):
        self.url_list = ['http://en.wikipedia.org/wiki/List_of_S%26P_500_companies','http://etfdb.com/compare/market-cap/']
        self.wiki = pd.read_html(requests.get(self.url_list[0], headers={'User-agent': 'Mozilla/5.0'}).text)
        self.dbetf = pd.read_html(requests.get(self.url_list[1], headers={'User-agent': 'Mozilla/5.0'}).text)
        self.stock_tickers = list(self.wiki[0][0].values[1:])
        self.etf_tickers = list(self.dbetf[0]['Symbol'])
        self.stock = self.get_df(self.stock_tickers)
        self.etf = self.get_df(self.etf_tickers)

    def mp_asset(self,ticker):
        data = pdr.get_data_yahoo(ticker, start='2015-04-01', end = '2018-04-01')
        return data

    def get_df(self,tickers):
        with Pool(4) as p:
            #'''each entry is a dict, key = ticker, value = dataframe'''
            data_mped = p.map(self.mp_asset, tickers)

        data_dict = dict(zip(tickers, data_mped))
        data_dict = { k:v for k,v in data_dict.items() if len(v) > 0 }

        data_adj_dict = {}
        for k in data_dict.keys():
            data_adj_dict[k] = data_dict[k]['Adj Close']
        data_adj_df = pd.DataFrame.from_dict(data_adj_dict).dropna(axis=1,how='any')
        return data_adj_df

ModuleNotFoundError: No module named 'fix_yahoo_finance'

In [81]:
data = combined5
daily_return_list =[]
for i in range(1,len(data)):
    daily_return_list.append(list(data.iloc[i]/data.iloc[i-1] -1))
daily_returns = pd.DataFrame(daily_return_list, columns=data.columns)


daily_return_sum = 0
for i in range(len(daily_returns)):
    daily_return_sum += daily_returns.iloc[i]
#annual_returns= (daily_returns.mean()+1) ** 252-1
daily_return_mean = daily_return_sum/len(daily_returns)
annual_returns= (daily_return_mean+1) ** 252-1
daily_diff_list = []
for i in range(len(daily_returns)):
    daily_diff_list.append(daily_returns.iloc[i] - daily_return_mean)
daily_diff = pd.DataFrame(daily_diff_list, columns=data.columns)

cov_lists = []
for j in range(len(daily_diff.columns)):
    cov_list =[]
    for k in range(len(daily_diff.columns)):
        diff_sum = 0
        for i in range(len(daily_diff)):
            diff_sum += daily_diff.iloc[i][j]*daily_diff.iloc[i][k]
        cov_list.append(diff_sum/(len(daily_diff)-1))
    cov_lists.append(cov_list)
daily_covariance= pd.DataFrame(cov_lists, columns=data.columns, index = data.columns)
annual_covariance = daily_covariance * 252
stocks_selected = list(data.columns)
#empty lists to store returns, volatility and weights of imiginary portfolios
returns_list = []
volatility_list = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = len(stocks_selected)
num_portfolios = 10000

In [86]:
weights = np.random.random(num_assets)
#weights /= np.sum(weights)
weights_sum = 0
for w in weights:
    weights_sum += w
weights /= weights_sum
returns =0
for i in range(len(weights)):
    returns += weights[i]*annual_returns[i]

In [88]:
returns

0.4109069317746073

In [84]:
annual_returns

btc    3.315230
spy    0.126030
tlt    0.014784
vxx   -0.424494
xlf    0.298959
fxi    0.096668
Name: 0, dtype: float64

In [89]:
# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    #weights /= np.sum(weights)
    weights_sum = 0
    for w in weights:
        weights_sum += w
    weights /= weights_sum

    #returns = np.dot(weights, annual_returns)
    returns =0
    for i in range(len(weights)):
        returns += weights[i]*annual_returns[i]
    #volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
    weighted_covariances = []
    for i in range(len(annual_covariance)):
        weighted_covariance =0 
        for j in range(len(weights)):
            weighted_covariance += annual_covariance.iloc[i][j]*weights[j]
        weighted_covariances.append(weighted_covariance)
    volatility = 0
    for i in range(len(weights)):
        volatility += weights[i]* weighted_covariances[i]

    #volatility = np.sqrt(multiply(weights.T,  multiply(annual_covariance, weights)))
    sharpe = (returns- 0.015) / volatility
    sharpe_ratio.append(sharpe)
    returns_list.append(returns)
    volatility_list.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': returns_list,
             'Volatility': volatility_list,
             'Sharpe Ratio': sharpe_ratio}
# extend original dictionary to accomodate each ticker and weight in the portfolio
for ct,ticker in enumerate(stocks_selected):
    portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

# basic 

In [91]:
def stock_simulate(data, interest_rate, volatility_given):
    # calculate daily and annual returns of the stocks
    #daily_returns = data.pct_change(1)
    #daily_returns = daily_returns.dropna(axis =0, how = 'all')
    daily_return_list =[]
    for i in range(1,len(data)):
        daily_return_list.append(list(data.iloc[i]/data.iloc[i-1] -1))
    daily_returns = pd.DataFrame(daily_return_list, columns=data.columns)
        
        
    daily_return_sum = 0
    for i in range(len(daily_returns)):
        daily_return_sum += daily_returns.iloc[i]
    #annual_returns= (daily_returns.mean()+1) ** 252-1
    daily_return_mean = daily_return_sum/len(daily_returns)
    annual_returns= (daily_return_mean+1) ** 252-1
    # get daily and covariance of returns of the stock
    #daily_covariance = daily_returns.cov()
    
    daily_diff_list = []
    for i in range(len(daily_returns)):
        daily_diff_list.append(daily_returns.iloc[i] - daily_return_mean)
    daily_diff = pd.DataFrame(daily_diff_list, columns=data.columns)

    cov_lists = []
    for j in range(len(daily_diff.columns)):
        cov_list =[]
        for k in range(len(daily_diff.columns)):
            diff_sum = 0
            for i in range(len(daily_diff)):
                diff_sum += daily_diff.iloc[i][j]*daily_diff.iloc[i][k]
            cov_list.append(diff_sum/(len(daily_diff)-1))
        cov_lists.append(cov_list)
    daily_covariance= pd.DataFrame(cov_lists, columns=data.columns, index = data.columns)
    
    annual_covariance = daily_covariance * 252
    stocks_selected = list(data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = []
    volatility_list = []
    sharpe_ratio = []
    stock_weights = []

    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        #weights /= np.sum(weights)
        weights_sum = 0
        for w in weights:
            weights_sum += w
        weights /= weights_sum
        
        #returns = np.dot(weights, annual_returns)
        returns =0
        for i in range(len(weights)):
            returns += weights[i]*annual_returns[i]
            
        
        
        #volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        weighted_covariances = []
        for i in range(len(annual_covariance)):
            weighted_covariance =0 
            for j in range(len(weights)):
                weighted_covariance += annual_covariance.iloc[i][j]*weights[j]
            weighted_covariances.append(weighted_covariance)
        volatility = 0
        for i in range(len(weights)):
            volatility += weights[i]* weighted_covariances[i]
            
        #volatility = np.sqrt(multiply(weights.T,  multiply(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility
        sharpe_ratio.append(sharpe)
        returns_list.append(returns)
        volatility_list.append(volatility)
        stock_weights.append(weights)

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': returns_list,
                 'Volatility': volatility_list,
                 'Sharpe Ratio': sharpe_ratio}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    #for ct,ticker in enumerate(stocks_selected):
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

            
            
    # make a nice dataframe of the extended dictionary
    portf = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    columns_na = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in stocks_selected]

    # reorder dataframe columns
    portf= portf[columns_na]

    best_df = portf[(portf.Volatility < volatility_given+0.001) & (portf.Volatility > volatility_given-0.01 )]
    #best_return = best_df.loc[best_df['Returns'].idxmax()]
    
    cur = 0
    max_index = 0
    for i in best_df['Returns'].index.values.tolist():
        if best_df['Returns'][i] > cur:
            cur = best_df['Returns'][i]
            max_index = i
    best_return = best_df.loc[max_index] 
    
    return best_return


In [92]:
stock_simulate(combined5, 0.015, 0.13)

Returns          1.709089
Volatility       0.130729
Sharpe Ratio    12.958751
btc Weight       0.494132
spy Weight       0.067931
tlt Weight       0.018635
vxx Weight       0.038811
xlf Weight       0.206562
fxi Weight       0.173929
Name: 8084, dtype: float64

In [107]:
data = combined5
daily_return_list =[]
for i in range(1,len(data)):
    daily_return_list.append(list(data.iloc[i]/data.iloc[i-1] -1))
daily_returns = pd.DataFrame(daily_return_list, columns=data.columns)

In [108]:
daily_returns

,btc,spy,tlt,vxx,xlf,fxi
0,0.004655,-0.016222,0.013158,0.030730,-0.021364,-0.023239
1,0.014518,-0.002342,0.007286,0.025254,0.006297,0.002453
2,-0.005920,0.012714,-0.000708,-0.059870,0.021694,0.010032
3,-0.022597,-0.006132,-0.003068,0.022562,-0.006941,0.005087
4,0.011142,0.013361,0.009627,-0.024555,0.012335,0.018077
5,-0.017320,-0.002972,0.008285,-0.004013,-0.002437,0.004261
6,-0.103231,0.012022,0.019302,-0.042857,0.005700,0.017445
7,0.019576,-0.004562,-0.005095,0.000000,-0.009312,-0.000232
8,0.003258,0.008827,0.005274,-0.015308,0.013577,0.004403
9,0.014058,-0.001949,-0.001216,-0.019044,-0.005666,-0.003230


In [113]:
for x in daily_returns.columns.values:
    daily_returns[x] = (1 + daily_returns[x]).cumprod() - 1


In [114]:
daily_returns.iloc[-1]

btc    36.016242
spy     0.391931
tlt     0.020784
vxx    -0.901414
xlf     1.012725
fxi     0.209428
Name: 753, dtype: float64

In [109]:
daily_returns.head()

,btc,spy,tlt,vxx,xlf,fxi
0,0.004655,-0.016222,0.013158,0.030730,-0.021364,-0.023239
1,0.014518,-0.002342,0.007286,0.025254,0.006297,0.002453
2,-0.005920,0.012714,-0.000708,-0.059870,0.021694,0.010032
3,-0.022597,-0.006132,-0.003068,0.022562,-0.006941,0.005087
4,0.011142,0.013361,0.009627,-0.024555,0.012335,0.018077


In [ ]:
def stock_simulate(data, interest_rate, volatility_given):
    # calculate daily and annual returns of the stocks
    #daily_returns = data.pct_change(1)
    #daily_returns = daily_returns.dropna(axis =0, how = 'all')
    daily_return_list =[]
    for i in range(1,len(data)):
        daily_return_list.append(list(data.iloc[i]/data.iloc[i-1] -1))
    daily_returns = pd.DataFrame(daily_return_list, columns=data.columns)
    
    for x in daily_returns.columns.values:
        daily_returns[x] = (1 + daily_returns[x]).cumprod() - 1
        
        
    daily_return_sum = 0
    for i in range(len(daily_returns)):
        daily_return_sum += daily_returns.iloc[i]
    daily_return_mean = daily_return_sum/len(daily_returns)
    annual_returns= (daily_return_mean+1) ** 252-1
    
    # get daily and covariance of returns of the stock
    #daily_covariance = daily_returns.cov()
    
    daily_diff_list = []
    for i in range(len(daily_returns)):
        daily_diff_list.append(daily_returns.iloc[i] - daily_return_mean)
    daily_diff = pd.DataFrame(daily_diff_list, columns=data.columns)

    cov_lists = []
    for j in range(len(daily_diff.columns)):
        cov_list =[]
        for k in range(len(daily_diff.columns)):
            diff_sum = 0
            for i in range(len(daily_diff)):
                diff_sum += daily_diff.iloc[i][j]*daily_diff.iloc[i][k]
            cov_list.append(diff_sum/(len(daily_diff)-1))
        cov_lists.append(cov_list)
    daily_covariance= pd.DataFrame(cov_lists, columns=data.columns, index = data.columns)
    
    annual_covariance = daily_covariance * 252
    stocks_selected = list(data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = []
    volatility_list = []
    sharpe_ratio = []
    stock_weights = []

    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        #weights /= np.sum(weights)
        weights_sum = 0
        for w in weights:
            weights_sum += w
        weights /= weights_sum
        
        #returns = np.dot(weights, annual_returns)
        returns =0
        for i in range(len(weights)):
            returns += weights[i]*annual_returns[i]
            
        
        
        #volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        weighted_covariances = []
        for i in range(len(annual_covariance)):
            weighted_covariance =0 
            for j in range(len(weights)):
                weighted_covariance += annual_covariance.iloc[i][j]*weights[j]
            weighted_covariances.append(weighted_covariance)
        volatility = 0
        for i in range(len(weights)):
            volatility += weights[i]* weighted_covariances[i]
            
        #volatility = np.sqrt(multiply(weights.T,  multiply(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility
        sharpe_ratio.append(sharpe)
        returns_list.append(returns)
        volatility_list.append(volatility)
        stock_weights.append(weights)

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': returns_list,
                 'Volatility': volatility_list,
                 'Sharpe Ratio': sharpe_ratio}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    #for ct,ticker in enumerate(stocks_selected):
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

            
            
    # make a nice dataframe of the extended dictionary
    portf = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    columns_na = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in stocks_selected]

    # reorder dataframe columns
    portf= portf[columns_na]

    best_df = portf[(portf.Volatility < volatility_given+0.001) & (portf.Volatility > volatility_given-0.01 )]
    #best_return = best_df.loc[best_df['Returns'].idxmax()]
    
    cur = 0
    max_index = 0
    for i in best_df['Returns'].index.values.tolist():
        if best_df['Returns'][i] > cur:
            cur = best_df['Returns'][i]
            max_index = i
    best_return = best_df.loc[max_index] 
    
    return best_return


In [95]:
import numpy as np

class Simulation(object):
    def __init__(self, data, int_rate, vol, tolerance):
        self.data = data
        self.int_rate = int_rate
        self.vol = vol
        self.tolerance = tolerance
        self.valid_index, self.portfolio, self.volatility_list = self.portNvalidIndex()
        self.best_return_portfolio, self.flag = self.best_return()
    
    def portNvalidIndex(self):
        # calculate daily and annual returns of the stocks
        
        daily_return_list =[]
        for i in range(1,len(self.data)):
            daily_return_list.append(list(self.data.iloc[i]/self.data.iloc[i-1] -1))
        daily_returns = pd.DataFrame(daily_return_list, columns=self.data.columns)


        daily_return_sum = 0
        for i in range(len(daily_returns)):
            daily_return_sum += daily_returns.iloc[i]
        daily_return_mean = daily_return_sum/len(daily_returns)
        annual_returns= (daily_return_mean+1) ** 252-1
        # get daily and covariance of returns of the stock
      

        daily_diff_list = []
        for i in range(len(daily_returns)):
            daily_diff_list.append(daily_returns.iloc[i] - daily_return_mean)
        daily_diff = pd.DataFrame(daily_diff_list, columns=self.data.columns)

        cov_lists = []
        for j in range(len(daily_diff.columns)):
            cov_list =[]
            for k in range(len(daily_diff.columns)):
                diff_sum = 0
                for i in range(len(daily_diff)):
                    diff_sum += daily_diff.iloc[i][j]*daily_diff.iloc[i][k]
                cov_list.append(diff_sum/(len(daily_diff)-1))
            cov_lists.append(cov_list)
        daily_covariance= pd.DataFrame(cov_lists, columns=self.data.columns, index = self.data.columns)

        annual_covariance = daily_covariance * 252
        stocks_selected = list(self.data.columns)
        #empty lists to store returns, volatility and weights of imiginary portfolios
        returns_list = []
        volatility_list = []
        sharpe_ratio = []
        stock_weights = []

        # set the number of combinations for imaginary portfolios
        num_assets = len(stocks_selected)
        num_portfolios = 10000

        # populate the empty lists with each portfolios returns,risk and weights
        for single_portfolio in range(num_portfolios):
            weights = np.random.random(num_assets)
            weights_sum = 0
            for w in weights:
                weights_sum += w
            weights /= weights_sum
            returns =0
            for i in range(len(weights)):
                returns += weights[i]*annual_returns[i]
            weighted_covariances = []
            for i in range(len(annual_covariance)):
                weighted_covariance =0 
                for j in range(len(weights)):
                    weighted_covariance += annual_covariance.iloc[i][j]*weights[j]
                weighted_covariances.append(weighted_covariance)
            volatility = 0
            for i in range(len(weights)):
                volatility += weights[i]* weighted_covariances[i]
            sharpe = (returns- self.int_rate) / volatility
            sharpe_ratio.append(sharpe)
            returns_list.append(returns)
            volatility_list.append(volatility)
            stock_weights.append(weights)

        # a dictionary for Returns and Risk values of each portfolio
        portfolio = {'Returns': returns_list,
                     'Volatility': volatility_list,
                     'Sharpe Ratio': sharpe_ratio}

        # extend original dictionary to accomodate each ticker and weight in the portfolio
        for ct,ticker in enumerate(stocks_selected):
            portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]
        valid_index = [x for x in range(len(portfolio['Volatility'])) 
                       if (portfolio['Volatility'][x] < self.vol+self.tolerance) 
                       & (portfolio['Volatility'][x] > self.vol-self.tolerance)]
        return valid_index, portfolio, volatility_list  
            
            
    def best_return(self):
        if len(self.valid_index) == 0:
            print('No matching volatility level.')
            if (max(self.volatility_list) + self.tolerance) < self.vol:
                flag = 0
                print('The maximum volatility of our portfolios is {} less than your input risk level'.format(self.vol - max(self.volatility_dict)))
            if (min(self.volatility_list) - self.tolerance) > self.vol:
                flag = 1
                print('The minimum volatility of our portfolios is {} greater than your input risk level'.format(min(self.volatility_dict) - self.vol))
            print('We will return the portafolio we found with the highest return.')
            best_index = self.portfolio['Returns'].index(np.max(self.portfolio['Returns']))
            best_return_portfolio = {'Returns': self.portfolio['Returns'][best_index],
                                     'Volatility':self.portfolio['Volatility'][best_index],
                                     'Sharpe Ratio':self.portfolio['Sharpe Ratio'][best_index]
                                    }

            stocks_selected = self.data.columns
            for ticker in list(self.data.columns):
                best_return_portfolio[ticker+' Weight'] = self.portfolio[ticker+' Weight'][best_index]
        else:
            flag = 2
            best_index = self.portfolio['Returns'].index(max([self.portfolio['Returns'][i] for i in self.valid_index]))
            best_return_portfolio = {'Returns': self.portfolio['Returns'][best_index],
                                     'Volatility':self.portfolio['Volatility'][best_index],
                                     'Sharpe Ratio':self.portfolio['Sharpe Ratio'][best_index]
                                    }

            stocks_selected = self.data.columns
            for ticker in list(self.data.columns):
                best_return_portfolio[ticker+' Weight'] = self.portfolio[ticker+' Weight'][best_index]

        return best_return_portfolio, flag   


In [96]:
Simulation(combined5, 0.015, 0.13, 0.01).best_return_portfolio

{'Returns': 1.7174589453322195,
 'Sharpe Ratio': 12.819262906444722,
 'Volatility': 0.1328047453084319,
 'btc Weight': 0.5039888568213942,
 'fxi Weight': 0.10495375225374733,
 'spy Weight': 0.0827958543643787,
 'tlt Weight': 0.12371837615370558,
 'vxx Weight': 0.04279466642861055,
 'xlf Weight': 0.14174849397816391}

# OPT1

In [19]:
def stock_simulate_opt1(data, interest_rate, volatility_given):
    # calculate daily and annual returns of the stocks
    daily_returns = data.pct_change(1)
    annual_returns= (daily_returns.mean()+1) ** 252-1
    # get daily and covariance of returns of the stock
    daily_covariance = daily_returns.cov()
    annual_covariance = daily_covariance * 252
    stocks_selected = list(data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = []
    volatility_list = []
    sharpe_ratio = []
    stock_weights = []
    

    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, annual_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility
        sharpe_ratio.append(sharpe)
        returns_list.append(returns)
        volatility_list.append(volatility)
        stock_weights.append(weights)

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': returns_list,
                 'Volatility': volatility_list,
                 'Sharpe Ratio': sharpe_ratio}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

    # make a nice dataframe of the extended dictionary
    portf = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    columns_na = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in stocks_selected]

    # reorder dataframe columns
    portf= portf[columns_na]

    best_df = portf[(portf.Volatility< volatility_given+0.001) & (portf.Volatility >volatility_given-0.01 )]
    best_return = best_df.loc[best_df['Returns'].idxmax()]
      
    
    return best_return

In [22]:
%timeit stock_simulate_opt1(combined5, 0.015, 0.16)

724 ms ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
stock_simulate_opt1(combined5, 0.015, 0.16)

Returns         0.759858
Volatility      0.160525
Sharpe Ratio    4.640127
btc Weight      0.202259
spy Weight      0.274522
tlt Weight      0.205765
vxx Weight      0.057232
xlf Weight      0.251230
fxi Weight      0.008992
Name: 9009, dtype: float64

# OPT2

In [14]:
def stock_simulate_opt2(data, interest_rate, volatility_given):
    # calculate daily and annual returns of the stocks
    daily_returns = data.pct_change(1)
    annual_returns= (daily_returns.mean()+1) ** 252-1
    # get daily and covariance of returns of the stock
    daily_covariance = daily_returns.cov()
    annual_covariance = daily_covariance * 252
    stocks_selected = list(data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = dict()
    volatility_list = dict()
    sharpe_ratio = dict()
    stock_weights = []
    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, annual_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility
        
        sharpe_ratio[sharpe] = sharpe
        returns_list[returns] = returns
        volatility_list[volatility]= volatility
        stock_weights.append(weights)
        

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': list(returns_list.values()),
                 'Volatility': list(volatility_list.values()),
                 'Sharpe Ratio': list(sharpe_ratio.values())}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

    # make a nice dataframe of the extended dictionary
    portf = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    columns_na = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in stocks_selected]

    # reorder dataframe columns
    portf= portf[columns_na]

    best_df = portf[(portf.Volatility< volatility_given+0.001) & (portf.Volatility >volatility_given-0.01 )]
    best_return = best_df.loc[best_df['Returns'].idxmax()]
      
    
    return best_return, portfolio

In [15]:
%timeit stock_simulate_opt2(combined5, 0.015, 0.13)

1.81 s ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# OPT3

In [ ]:
import cython


In [ ]:
%load_ext Cython


In [23]:
%%cython
import numpy as np
import pandas as pd
def stock_simulate_opt3(data, interest_rate, volatility_given):
    # calculate daily and annual returns of the stocks
    daily_returns = data.pct_change(1)
    annual_returns= (daily_returns.mean()+1) ** 252-1
    # get daily and covariance of returns of the stock
    daily_covariance = daily_returns.cov()
    annual_covariance = daily_covariance * 252
    stocks_selected = list(data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = dict()
    volatility_list = dict()
    sharpe_ratio = dict()
    stock_weights = []
    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, annual_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility
        
        sharpe_ratio[sharpe] = sharpe
        returns_list[returns] = returns
        volatility_list[volatility]= volatility
        stock_weights.append(weights)
        

    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': list(returns_list.values()),
                 'Volatility': list(volatility_list.values()),
                 'Sharpe Ratio': list(sharpe_ratio.values())}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

    # make a nice dataframe of the extended dictionary
    portf = pd.DataFrame(portfolio)

    # get better labels for desired arrangement of columns
    columns_na = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock+' Weight' for stock in stocks_selected]

    # reorder dataframe columns
    portf= portf[columns_na]

    best_df = portf[(portf.Volatility< volatility_given+0.001) & (portf.Volatility >volatility_given-0.01 )]
    best_return = best_df.loc[best_df['Returns'].idxmax()]
      
    
    return best_return, portfolio

In [24]:
%timeit stock_simulate_opt3(combined5, 0.015, 0.13)

AttributeError: 'numpy.ndarray' object has no attribute 'pct_change'

In [16]:
data = combined5
data= data.as_matrix()
daily_diff = np.diff(data, axis =0)
daily_returns = np.divide(daily_diff, data[:-1])
annual_returns = (daily_returns.mean(axis =0)+1)**252 -1
m,n = daily_returns.shape
daily_covariance = np.zeros([n,n])

In [17]:
daily_covariance

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [28]:
import itertools
daily_ret_ind = list(itertools.product(range(n), range(n)))

for i, j in daily_ret_ind:
    print(i,j)

In [30]:
for i, j in daily_ret_ind:
    daily_covariance[i,j] =np.cov(daily_returns[:,i], daily_returns[:,j])[0][1]

In [117]:
data = combined5
data= data.as_matrix()
data.pct_change(1)

AttributeError: 'numpy.ndarray' object has no attribute 'pct_change'

In [18]:
combined5 = combined5.as_matrix()

# OPT4

In [21]:
import cython
%load_ext Cython

In [22]:
%%cython
import itertools
import numpy

def stock_simulate_opt4(data, interest_rate, volatility_given):
    #data= data.as_matrix()
    daily_diff = np.diff(data, axis =0)
    daily_returns = np.divide(daily_diff, data[:-1])
    annual_returns = (daily_returns.mean(axis =0)+1)**252 -1
    
    m,n = daily_returns.shape
    daily_covariance = np.zeros([n,n])
    for i, j in daily_ret_ind:
        daily_covariance[i,j] =np.cov(daily_returns[:,i], daily_returns[:,j])[0][1]
    annual_covariance = daily_covariance * 252

    stocks_selected = list(combined5.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_list = dict()
    volatility_list = dict()
    sharpe_ratio = dict()
    stock_weights = []
    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 10000
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, annual_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        sharpe = (returns- interest_rate) / volatility

        sharpe_ratio[sharpe] = sharpe
        returns_list[returns] = returns
        volatility_list[volatility]= volatility
        stock_weights.append(weights)
    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': list(returns_list.values()),
                 'Volatility': list(volatility_list.values()),
                 'Sharpe Ratio': list(sharpe_ratio.values())}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]
    
    valid_index = [x for x in range(len(portfolio['Volatility'])) if (portfolio['Volatility'][x]< volatility_given+0.001) & (portfolio['Volatility'][x] >volatility_given-0.01  )  ]
    
    return best_return_portfolio

    



Error compiling Cython file:
------------------------------------------------------------
...
import itertools
import numpy

def stock_simulate_opt4(data, interest_rate, volatility_given):
    #data= data.as_matrix()
    daily_diff = np.diff(data, axis =0)
                ^
------------------------------------------------------------

C:\Users\sherryyang\.ipython\cython\_cython_magic_add5dc73b44838ed6ab06ad01c498fb6.pyx:6:17: undeclared name not builtin: np

Error compiling Cython file:
------------------------------------------------------------
...
    daily_returns = np.divide(daily_diff, data[:-1])
    annual_returns = (daily_returns.mean(axis =0)+1)**252 -1
    
    m,n = daily_returns.shape
    daily_covariance = np.zeros([n,n])
    for i, j in daily_ret_ind:
               ^
------------------------------------------------------------

C:\Users\sherryyang\.ipython\cython\_cython_magic_add5dc73b44838ed6ab06ad01c498fb6.pyx:12:16: undeclared name not builtin: daily_ret_ind

Error 

TypeError: object of type 'NoneType' has no len()

In [20]:
%timeit stock_simulate_opt4(combined5, 0.015, 0.3)

AssertionError: Failed at object (object mode frontend)


In [33]:
stock_simulate_opt4(combined5, 0.015, 0.3)

{'Returns': 1.4228215176144319,
 'Sharpe Ratio': 4.72233262763757,
 'Volatility': 0.2981199395771321,
 'btc Weight': 0.3980421287042417,
 'fxi Weight': 0.06055110262493215,
 'spy Weight': 0.02236853100173828,
 'tlt Weight': 0.1644790756811929,
 'vxx Weight': 0.019188561582794462,
 'xlf Weight': 0.3353706004051006}

In [13]:
 @jit(debug=True)
def portNvalidIndex(self):
    data= self.data.as_matrix()
    daily_diff = np.diff(data, axis =0)
    daily_returns = np.divide(daily_diff, data[:-1])
    annual_returns = (daily_returns.mean(axis =0)+1)**252 -1
    m,n = daily_returns.shape
    daily_covariance = np.zeros([n,n])
    daily_ret_ind = list(itertools.product(range(n), range(n)))
    for i, j in daily_ret_ind:
        daily_covariance[i,j] =np.cov(daily_returns[:,i], daily_returns[:,j])[0][1]
    annual_covariance = daily_covariance * 252
    stocks_selected = list(self.data.columns)
    #empty lists to store returns, volatility and weights of imiginary portfolios
    returns_dict = dict()
    volatility_dict = dict()
    sharpe_ratio = dict()
    stock_weights = []
    # set the number of combinations for imaginary portfolios
    num_assets = len(stocks_selected)
    num_portfolios = 50000
    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, annual_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
        sharpe = (returns - self.int_rate) / volatility

        sharpe_ratio[sharpe] = sharpe
        returns_dict[returns] = returns
        volatility_dict[volatility]= volatility
        stock_weights.append(weights)
    # a dictionary for Returns and Risk values of each portfolio
    portfolio = {'Returns': list(returns_dict.values()),
                 'Volatility': list(volatility_dict.values()),
                 'Sharpe Ratio': list(sharpe_ratio.values())}

    # extend original dictionary to accomodate each ticker and weight in the portfolio
    for ct,ticker in enumerate(stocks_selected):
        portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

    valid_index = [x for x in range(len(portfolio['Volatility'])) 
                   if (portfolio['Volatility'][x] < self.vol+self.tolerance) 
                   & (portfolio['Volatility'][x] > self.vol-self.tolerance)]
    return valid_index, portfolio, volatility_dict

In [14]:
import numpy as np
import itertools


class Simulation(object):
    def __init__(self, data, int_rate, vol, tolerance):
        self.data = data
        self.int_rate = int_rate
        self.vol = vol
        self.tolerance = tolerance
        self.valid_index, self.portfolio, self.volatility_dict = self.portNvalidIndex()
        self.best_return_portfolio, self.flag = self.best_return()
    @jit(debug=True)
    def portNvalidIndex(self):
        data= self.data.as_matrix()
        daily_diff = np.diff(data, axis =0)
        daily_returns = np.divide(daily_diff, data[:-1])
        annual_returns = (daily_returns.mean(axis =0)+1)**252 -1
        m,n = daily_returns.shape
        daily_covariance = np.zeros([n,n])
        daily_ret_ind = list(itertools.product(range(n), range(n)))
        for i, j in daily_ret_ind:
            daily_covariance[i,j] =np.cov(daily_returns[:,i], daily_returns[:,j])[0][1]
        annual_covariance = daily_covariance * 252
        stocks_selected = list(self.data.columns)
        #empty lists to store returns, volatility and weights of imiginary portfolios
        returns_dict = dict()
        volatility_dict = dict()
        sharpe_ratio = dict()
        stock_weights = []
        # set the number of combinations for imaginary portfolios
        num_assets = len(stocks_selected)
        num_portfolios = 50000
        for single_portfolio in range(num_portfolios):
            weights = np.random.random(num_assets)
            weights /= np.sum(weights)
            returns = np.dot(weights, annual_returns)
            volatility = np.sqrt(np.dot(weights.T, np.dot(annual_covariance, weights)))
            sharpe = (returns - self.int_rate) / volatility

            sharpe_ratio[sharpe] = sharpe
            returns_dict[returns] = returns
            volatility_dict[volatility]= volatility
            stock_weights.append(weights)
        # a dictionary for Returns and Risk values of each portfolio
        portfolio = {'Returns': list(returns_dict.values()),
                     'Volatility': list(volatility_dict.values()),
                     'Sharpe Ratio': list(sharpe_ratio.values())}

        # extend original dictionary to accomodate each ticker and weight in the portfolio
        for ct,ticker in enumerate(stocks_selected):
            portfolio[ticker+' Weight'] = [Weight[ct] for Weight in stock_weights]

        valid_index = [x for x in range(len(portfolio['Volatility'])) 
                       if (portfolio['Volatility'][x] < self.vol+self.tolerance) 
                       & (portfolio['Volatility'][x] > self.vol-self.tolerance)]
        return valid_index, portfolio, volatility_dict
    
    def best_return(self):
        if len(self.valid_index) == 0:
            print('No matching volatility level.')
            if (max(self.volatility_dict) + self.tolerance) < self.vol:
                flag = 0
                print('The maximum volatility of our portfolios is {} less than your input risk level'.format(self.vol - max(self.volatility_dict)))
            if (min(self.volatility_dict) - self.tolerance) > self.vol:
                flag = 1
                print('The minimum volatility of our portfolios is {} greater than your input risk level'.format(min(self.volatility_dict) - self.vol))
            print('We will return the portafolio we found with the highest return.')
            best_index = self.portfolio['Returns'].index(np.max(self.portfolio['Returns']))
            best_return_portfolio = {'Returns': self.portfolio['Returns'][best_index],
                                     'Volatility':self.portfolio['Volatility'][best_index],
                                     'Sharpe Ratio':self.portfolio['Sharpe Ratio'][best_index]
                                    }
            
            stocks_selected = self.data.columns
            for ticker in list(self.data.columns):
                best_return_portfolio[ticker+' Weight'] = self.portfolio[ticker+' Weight'][best_index]
        else:
            flag = 2
            best_index = self.portfolio['Returns'].index(max([self.portfolio['Returns'][i] for i in self.valid_index]))
            best_return_portfolio = {'Returns': self.portfolio['Returns'][best_index],
                                     'Volatility':self.portfolio['Volatility'][best_index],
                                     'Sharpe Ratio':self.portfolio['Sharpe Ratio'][best_index]
                                    }
            
            stocks_selected = self.data.columns
            for ticker in list(self.data.columns):
                best_return_portfolio[ticker+' Weight'] = self.portfolio[ticker+' Weight'][best_index]
                
        return best_return_portfolio, flag



In [15]:
Simulation(combined5, 0.015, 0.13, 0.01).best_return_portfolio

AssertionError: Failed at object (object mode frontend)


In [40]:
Simulation(combined5, 0.015, 0.13, 0.01).portfolio

{'Returns': [0.9400732046148415,
  0.4165512384791792,
  0.3882243981885174,
  1.12238162751475,
  0.7649563938356952,
  0.6743101374141398,
  1.6716755890285135,
  0.40559757137934993,
  0.5093202263771811,
  0.7112057771430145,
  0.7081440721004706,
  0.7500833028432712,
  0.12999770267146024,
  0.8665641915927945,
  0.10315994920942224,
  0.04420886569643199,
  0.6910923087452499,
  0.32270476525868963,
  0.21226406787227675,
  0.8938321566262533,
  0.13604379971657563,
  0.5088545103658514,
  0.44061562433057394,
  0.13354456536234602,
  0.7743901565569453,
  0.3700231931345541,
  0.608345627547014,
  0.5174136671787622,
  0.828555031026507,
  0.5976248917223344,
  1.0206709296415752,
  0.665895541066513,
  0.5333868251983814,
  0.3800066576343123,
  0.4529948796361097,
  0.8749702450577154,
  0.3292179096521531,
  1.5046509095597804,
  0.9114160636963815,
  0.16323457718713388,
  -0.012382972389477803,
  0.6460824244811182,
  0.4391687062140392,
  0.21913338758897166,
  0.06671202

In [41]:
Simulation(combined5, 0.015, 0.13, 0.01).volatility_dict

{0.17447339506642107: 0.17447339506642107,
 0.17711368041930808: 0.17711368041930808,
 0.16020135515365794: 0.16020135515365794,
 0.1597536031643843: 0.1597536031643843,
 0.09135131091606906: 0.09135131091606906,
 0.14043907912309728: 0.14043907912309728,
 0.22022388179717708: 0.22022388179717708,
 0.11104026854800396: 0.11104026854800396,
 0.16476536655345736: 0.16476536655345736,
 0.17908732429467766: 0.17908732429467766,
 0.17045741569543746: 0.17045741569543746,
 0.11743339377882395: 0.11743339377882395,
 0.13424279948627335: 0.13424279948627335,
 0.2809757974996054: 0.2809757974996054,
 0.13998146560029248: 0.13998146560029248,
 0.17208465125593148: 0.17208465125593148,
 0.23438337465093087: 0.23438337465093087,
 0.13702440921281403: 0.13702440921281403,
 0.09224018282213681: 0.09224018282213681,
 0.16304489177661305: 0.16304489177661305,
 0.09940732831412648: 0.09940732831412648,
 0.25618067658568655: 0.25618067658568655,
 0.12585403569914705: 0.12585403569914705,
 0.175245132642